In [80]:
import numpy as np
import pandas as pd
import datetime  # For datetime objects
import time
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt
from backtrader.feeds import PandasData

class PandasData_Signal(PandasData):
    lines = ('Predicted',)
    params = (('Predicted', 5), )

# Create a Stratey
class TestStrategy(bt.Strategy):
    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries

        self.datasignal = self.datas[0].Predicted
        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None
        self.cashrecord = pd.DataFrame(['Date', 'Value'])

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
        
    def get_cash_record(self):
        try:
            self.cashrecord = self.cashrecord.drop(0, axis=1)
        except:
            pass
        
        return self.cashrecord

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.datasignal[0])
        self.cashrecord = self.cashrecord.append({'Date': self.datas[0].datetime.datetime(0) , 'Value': self.broker.get_cash()}, ignore_index=True)

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return
        
        try:
            # Check if we are in the market
            if not self.position:

                # Not yet ... we MIGHT BUY if ...
                if self.datasignal[1] > self.datasignal[0]:
                    # BUY, BUY, BUY!!! (with default parameters)
                    self.log('BUY CREATE, %.2f' % self.datasignal[0])

                    # Keep track of the created order to avoid a 2nd order
                    self.order = self.buy()

            else:
                # Already in the market ... we might sell
                if self.datasignal[1] < self.datasignal[0]:
                    # SELL, SELL, SELL!!! (with all possible default parameters)
                    self.log('SELL CREATE, %.2f' % self.datasignal[0])
                    print(self.log('Current Cash {}'.format(self.broker.get_cash())))
                    # Keep track of the created order to avoid a 2nd order
                    self.order = self.sell()
        except:
            pass

if __name__ == '__main__':
    
    df = pd.read_csv('HB_HOUSTON.csv',  encoding='utf-8')[:1000]
    df = df.drop('Unnamed: 0', axis=1)
    np.random.seed(1)
    df['Date'] = [int(time.mktime(datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S').timetuple())) for x in df['Date']] 
    df['Datetime'] = [datetime.datetime.fromtimestamp(x) for x in df['Date']]
    df = df.drop('Date', axis=1)
    df['Signal'] = np.random.rand(df.shape[0])
    df['Open'] = df['SettlementPointPrice']
    df['High'] = df['SettlementPointPrice']
    df['Low'] = df['SettlementPointPrice']
    df['Close'] = df['SettlementPointPrice']
    df['Predicted'] = ((df['Predicted'] - df['Open'])/df['Open']) * 100
    df = df[['Datetime', 'Open', 'High', 'Low', 'Close', 'Predicted']]
    
    data = PandasData_Signal(dataname=df, datetime=0 ,open=1, high=2, low=3, close=4, Predicted=5)
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    cerebro.addstrategy(TestStrategy)


    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)

    # Add a FixedSize sizer according to the stake
    cerebro.addsizer(bt.sizers.FixedSize, stake=10)

    # Set the commission - 0.1% ... divide by 100 to remove the %
    cerebro.broker.setcommission(commission=0.001)

    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    rr = cerebro.run()

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
2010-12-01, Close, nan
2010-12-01, Close, nan
2010-12-01, Close, nan
2010-12-01, Close, nan
2010-12-01, Close, nan
2010-12-01, Close, nan
2010-12-01, Close, nan
2010-12-01, Close, nan
2010-12-01, Close, nan
2010-12-01, Close, nan
2010-12-01, Close, nan
2010-12-01, Close, nan
2010-12-01, Close, nan
2010-12-01, Close, nan
2010-12-01, Close, nan
2010-12-01, Close, nan
2010-12-01, Close, nan
2010-12-01, Close, nan
2010-12-01, Close, nan
2010-12-01, Close, nan
2010-12-01, Close, nan
2010-12-01, Close, nan
2010-12-01, Close, 18.51
2010-12-01, Close, 16.80
2010-12-01, Close, 14.15
2010-12-01, BUY CREATE, 14.15
2010-12-01, BUY EXECUTED, Price: 33.86, Cost: 338.60, Comm 0.34
2010-12-01, Close, 15.59
2010-12-01, SELL CREATE, 15.59
2010-12-01, Current Cash 99661.06139999999
None
2010-12-01, SELL EXECUTED, Price: 44.84, Cost: 338.60, Comm 0.45
2010-12-01, OPERATION PROFIT, GROSS 109.80, NET 109.01
2010-12-01, Close, 11.56
2010-12-01, Close, 4.05
2010-12-01, Clos

2010-12-02, Close, 13.51
2010-12-02, Close, 10.43
2010-12-02, Close, 8.74
2010-12-02, Close, 7.82
2010-12-02, Close, 7.61
2010-12-02, Close, 6.92
2010-12-02, Close, 6.50
2010-12-02, Close, 5.95
2010-12-02, Close, 5.69
2010-12-02, BUY CREATE, 5.69
2010-12-02, BUY EXECUTED, Price: 27.53, Cost: 275.30, Comm 0.28
2010-12-02, Close, 6.26
2010-12-02, Close, 6.79
2010-12-02, Close, 8.18
2010-12-02, Close, 8.91
2010-12-02, Close, 9.28
2010-12-02, SELL CREATE, 9.28
2010-12-02, Current Cash 99703.88669999996
None
2010-12-02, SELL EXECUTED, Price: 28.44, Cost: 275.30, Comm 0.28
2010-12-02, OPERATION PROFIT, GROSS 9.10, NET 8.54
2010-12-02, Close, 8.09
2010-12-02, Close, 7.44
2010-12-02, Close, 6.97
2010-12-02, BUY CREATE, 6.97
2010-12-02, BUY EXECUTED, Price: 28.27, Cost: 282.70, Comm 0.28
2010-12-02, Close, 7.71
2010-12-02, Close, 9.08
2010-12-02, Close, 11.73
2010-12-02, Close, 14.56
2010-12-02, Close, 16.34
2010-12-02, SELL CREATE, 16.34
2010-12-02, Current Cash 99705.01959999996
None
2010-12-

2010-12-04, SELL CREATE, 22.93
2010-12-04, Current Cash 99684.60500000005
None
2010-12-04, SELL EXECUTED, Price: 17.31, Cost: 195.70, Comm 0.17
2010-12-04, OPERATION PROFIT, GROSS -22.60, NET -22.97
2010-12-04, Close, 20.67
2010-12-04, BUY CREATE, 20.67
2010-12-04, BUY EXECUTED, Price: 15.72, Cost: 157.20, Comm 0.16
2010-12-04, Close, 21.94
2010-12-04, Close, 23.51
2010-12-04, Close, 27.86
2010-12-04, SELL CREATE, 27.86
2010-12-04, Current Cash 99700.17470000006
None
2010-12-04, SELL EXECUTED, Price: 16.08, Cost: 157.20, Comm 0.16
2010-12-04, OPERATION PROFIT, GROSS 3.60, NET 3.28
2010-12-04, Close, 21.71
2010-12-04, BUY CREATE, 21.71
2010-12-04, BUY EXECUTED, Price: 10.58, Cost: 105.80, Comm 0.11
2010-12-04, Close, 38.51
2010-12-04, Close, 72.58
2010-12-04, Close, 150.16
2010-12-04, SELL CREATE, 150.16
2010-12-04, Current Cash 99754.90810000006
None
2010-12-04, SELL EXECUTED, Price: 6.47, Cost: 105.80, Comm 0.06
2010-12-04, OPERATION PROFIT, GROSS -41.10, NET -41.27
2010-12-04, Close,

2010-12-05, SELL CREATE, 41.59
2010-12-05, Current Cash 100281.84320000003
None
2010-12-05, SELL EXECUTED, Price: 32.40, Cost: 325.40, Comm 0.32
2010-12-05, OPERATION PROFIT, GROSS -1.40, NET -2.05
2010-12-05, Close, 35.38
2010-12-05, Close, 35.00
2010-12-05, BUY CREATE, 35.00
2010-12-05, BUY EXECUTED, Price: 28.67, Cost: 286.70, Comm 0.29
2010-12-05, Close, 39.80
2010-12-05, Close, 40.29
2010-12-05, SELL CREATE, 40.29
2010-12-05, Current Cash 100318.53250000004
None
2010-12-05, SELL EXECUTED, Price: 28.80, Cost: 286.70, Comm 0.29
2010-12-05, OPERATION PROFIT, GROSS 1.30, NET 0.73
2010-12-05, Close, 35.27
2010-12-05, Close, 29.26
2010-12-05, Close, 26.28
2010-12-05, BUY CREATE, 26.28
2010-12-05, BUY EXECUTED, Price: 28.27, Cost: 282.70, Comm 0.28
2010-12-05, Close, 30.06
2010-12-05, Close, 35.09
2010-12-05, Close, 45.74
2010-12-05, Close, 55.77
2010-12-05, Close, 58.33
2010-12-05, SELL CREATE, 58.33
2010-12-05, Current Cash 100323.26180000005
None
2010-12-05, SELL EXECUTED, Price: 26.9

2010-12-07, BUY CREATE, 38.05
2010-12-07, BUY EXECUTED, Price: 28.34, Cost: 283.40, Comm 0.28
2010-12-07, Close, 40.66
2010-12-07, SELL CREATE, 40.66
2010-12-07, Current Cash 100770.55750000008
None
2010-12-07, SELL EXECUTED, Price: 27.81, Cost: 283.40, Comm 0.28
2010-12-07, OPERATION PROFIT, GROSS -5.30, NET -5.86
2010-12-07, Close, 40.02
2010-12-07, Close, 38.64
2010-12-07, Close, 35.09
2010-12-07, Close, 34.05
2010-12-07, Close, 31.88
2010-12-07, Close, 26.77
2010-12-07, Close, 22.76
2010-12-07, Close, 20.71
2010-12-07, Close, 18.24
2010-12-07, Close, 16.48
2010-12-07, Close, 14.13
2010-12-07, Close, 12.90
2010-12-07, Close, 12.11
2010-12-07, BUY CREATE, 12.11
2010-12-07, BUY EXECUTED, Price: 26.06, Cost: 260.60, Comm 0.26
2010-12-07, Close, 12.13
2010-12-07, SELL CREATE, 12.13
2010-12-07, Current Cash 100787.51880000009
None
2010-12-07, SELL EXECUTED, Price: 26.14, Cost: 260.60, Comm 0.26
2010-12-07, OPERATION PROFIT, GROSS 0.80, NET 0.28
2010-12-07, Close, 11.76
2010-12-07, BUY CR

2010-12-08, SELL CREATE, 79.58
2010-12-08, Current Cash 100312.95590000003
None
2010-12-08, SELL EXECUTED, Price: 28.97, Cost: 295.00, Comm 0.29
2010-12-08, OPERATION PROFIT, GROSS -5.30, NET -5.88
2010-12-08, Close, 75.88
2010-12-08, Close, 74.30
2010-12-08, BUY CREATE, 74.30
2010-12-08, BUY EXECUTED, Price: 29.28, Cost: 292.80, Comm 0.29
2010-12-08, Close, 75.08
2010-12-08, Close, 76.78
2010-12-08, Close, 78.59
2010-12-08, Close, 82.29
2010-12-08, Close, 92.15
2010-12-08, Close, 99.76
2010-12-08, SELL CREATE, 99.76
2010-12-08, Current Cash 100309.27340000003
None
2010-12-08, SELL EXECUTED, Price: 28.89, Cost: 292.80, Comm 0.29
2010-12-08, OPERATION PROFIT, GROSS -3.90, NET -4.48
2010-12-08, Close, 74.50
2010-12-08, Close, 65.08
2010-12-08, BUY CREATE, 65.08
2010-12-08, BUY EXECUTED, Price: 28.32, Cost: 283.20, Comm 0.28
2010-12-08, Close, 77.46
2010-12-08, Close, 84.07
2010-12-08, SELL CREATE, 84.07
2010-12-08, Current Cash 100314.40130000003
None
2010-12-08, SELL EXECUTED, Price: 28

2010-12-10, Close, 57.99
2010-12-10, Close, 58.90
2010-12-10, Close, 59.32
2010-12-10, Close, 60.69
2010-12-10, Close, 61.66
2010-12-10, SELL CREATE, 61.66
2010-12-10, Current Cash 102518.7361
None
2010-12-10, SELL EXECUTED, Price: 29.86, Cost: 298.30, Comm 0.30
2010-12-10, OPERATION PROFIT, GROSS 0.30, NET -0.30
2010-12-10, Close, 61.45
2010-12-10, Close, 59.29
2010-12-10, Close, 57.73
2010-12-10, Close, 53.08
2010-12-10, BUY CREATE, 53.08
2010-12-10, BUY EXECUTED, Price: 32.56, Cost: 325.60, Comm 0.33
2010-12-10, Close, 58.85
2010-12-10, Close, 61.56
2010-12-10, Close, 66.66
2010-12-10, Close, 74.55
2010-12-10, SELL CREATE, 74.55
2010-12-10, Current Cash 102491.1119
None
2010-12-10, SELL EXECUTED, Price: 43.43, Cost: 325.60, Comm 0.43
2010-12-10, OPERATION PROFIT, GROSS 108.70, NET 107.94
2010-12-10, Close, 64.60
2010-12-10, Close, 58.03
2010-12-10, Close, 46.88
2010-12-10, Close, -88.13
2010-12-10, BUY CREATE, -88.13
2010-12-10, BUY EXECUTED, Price: 111.09, Cost: 1110.90, Comm 1.11


In [81]:
d = rr[0].get_cash_record()

In [82]:
d

,Date,Value
0,NaT,NaN
1,NaT,NaN
2,2010-12-01 01:00:00,100000.0000
3,2010-12-01 01:15:00,100000.0000
4,2010-12-01 01:30:00,100000.0000
5,2010-12-01 01:45:00,100000.0000
6,2010-12-01 02:00:00,100000.0000
7,2010-12-01 02:15:00,100000.0000
8,2010-12-01 02:30:00,100000.0000
9,2010-12-01 02:45:00,100000.0000
